In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

import ROOT
import pdf_definitions as pdfs
import fitting_tools as ft

In [ ]:
BNC_tag = ""
BNC_bool = False

#BNC_tag = "_BNC"
#BNC_bool = True

#ntrain_tag = 'nsig_30000_nbkg_30000'
#ntrain_tag = 'nsig_40000_nbkg_40000_trial0'
#ntrain_tag = 'nsig_40000_nbkg_40000_trial1'
#ntrain_tag = 'features_2_nsig_40000_nbkg_40000_trial0'
#ntrain_tag = 'features_4_nsig_30000_nbkg_30000_trial4'
ntrain_tag = 'features_4_nsig_30000_nbkg_30000_trial15'

# THIS BNC SEEMS TO GIVE A NON-ZERO SIGNAL?
#ntrain_tag = 'features_4_nsig_30000_nbkg_30000_trial4'

df_sp, df_col, df_fom, info = ft.return_dataframes_and_info(BNC_tag=BNC_tag, BNC_bool=BNC_bool, ntrain_tag=ntrain_tag)


In [ ]:
x,y,ds,ds_x = ft.prepare_dataset(df_sp, max_proba_cut=info['max_proba_cut'], \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                             SIG_BKG_DATA='bkg')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

In [ ]:
#region_definitions

# Background


In [ ]:
'''
def fit_background(pars_d, sub_funcs, ds_x, verbose=False, pars_stored=None, FIT_SIDEBAND_ONLY=False):

    nentries = ds_x.sumEntries()
    print(f"nentries: {nentries}")
    reduced_data_x = None
    if FIT_SIDEBAND_ONLY:
        reduced_data = ds.reduce(ROOT.RooFit.CutRange("SB3"))
        reduced_data_x = reduced_data.reduce(x)
        nentries = reduced_data_x.sumEntries()
    print(f"nentries: {nentries}")

    pars_d["nbkg"].setVal(nentries)
    pars_d["nbkg"].setConstant(False)
    
    pars_d["argpar"].setVal(-23.5)
    pars_d["argpar"].setConstant(False)
    
    pars_d["cutoff"].setVal(5.29)
    if FIT_SIDEBAND_ONLY:
        pars_d["cutoff"].setConstant(True)
    else:
        pars_d["cutoff"].setConstant(True)
 
    # Bkg only
    total = ROOT.RooExtendPdf("fit_func", "Extended function for background", sub_funcs["bkg_pdf"], pars_d["nbkg"])

    if verbose:
        total.Print("v")

    fit_results = None
    if FIT_SIDEBAND_ONLY:
        fit_results = total.fitTo(reduced_data_x,  Save=True, PrintLevel=-1, Range='SB3')
    else:
        fit_results = total.fitTo(ds_x,  Save=True, PrintLevel=-1)

    if pars_stored is None:
        pars_stored = {}
    pars_stored['argpar'] = {'val':pars_d['argpar'].getVal(), 'err':pars_d['argpar'].getError()}
    pars_stored['cutoff'] = {'val':pars_d['cutoff'].getVal(), 'err':pars_d['cutoff'].getError()}
    
    print("\nFrom fit to background SP....\n")
    for val in pars_stored.keys():
        print(f"{val:12s}: {pars_stored[val]['val']:8.4f} +/- {pars_stored[val]['err']:.4f}")
    print()
    
    fit_results.Print("v")

    return pars_stored, total, fit_results, reduced_data_x
''';

In [ ]:
'''
def plot_background_after_fit(x, ds_x, total, sub_funcs, FIT_SIDEBAND_ONLY=False):
    xframe_bkg = x.frame(ROOT.RooFit.Title("Background only pdf with data"),   ROOT.RooFit.Bins(50))  # RooPlot
    
    c1 = ROOT.TCanvas("fit_bkg_only", "fit_bkg_only", 600, 300)#"fit", "fit", 600, 600)
    
    ds_x.plotOn(xframe_bkg)

    if FIT_SIDEBAND_ONLY:
        total.plotOn(xframe_bkg, ROOT.RooFit.Normalization(1.0))#, ROOT.RooAbsReal.RelativeExpected))
        
        # Overlay the background component of model with a dashed line
        ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
        total.plotOn(
            xframe_bkg, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
                ROOT.kDashed), ROOT.RooFit.Normalization(
                    1.0))#, ROOT.RooAbsReal.RelativeExpected))
    else:
        total.plotOn(xframe_bkg, ROOT.RooFit.Normalization(1.0, ROOT.RooAbsReal.RelativeExpected))
        
        # Overlay the background component of model with a dashed line
        ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
        total.plotOn(
            xframe_bkg, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
                ROOT.kDashed), ROOT.RooFit.Normalization(
                    1.0, ROOT.RooAbsReal.RelativeExpected))
    
    
    xframe_bkg.Draw()
    c1.Draw()

    return c1,xframe_bkg,ras_bkg
''';

In [ ]:
proba_cut = info['max_proba_cut']
#proba_cut = 0.0

x,y,ds,ds_x = ft.prepare_dataset(df_sp, max_proba_cut=proba_cut, \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                             SIG_BKG_DATA='bkg')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

FIT_SIDEBAND_ONLY=True

pars_stored = {}
pars_stored, total, fit_results_bkg, reduced_data_x = ft.fit_background(x, pars_d, sub_funcs, ds, ds_x, pars_stored,\
                                                                     FIT_SIDEBAND_ONLY=FIT_SIDEBAND_ONLY)

c1,xframe_bkg,ras_bkg = ft.plot_background_after_fit(x,reduced_data_x,total,sub_funcs, FIT_SIDEBAND_ONLY=True)


In [ ]:
nvalues = int(reduced_data_x.sumEntries())

print(nvalues)

for n in range(nvalues):
    values = reduced_data_x.get(n)
    print(values[0].getVal())

In [ ]:
proba_cut = info['max_proba_cut']
#proba_cut = 0.0

x,y,ds,ds_x = ft.prepare_dataset(df_sp, max_proba_cut=proba_cut, \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                             SIG_BKG_DATA='bkg')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

FIT_SIDEBAND_ONLY=False

pars_stored = {}
pars_stored, total, fit_results_bkg, reduced_data_x = ft.fit_background(x, pars_d, sub_funcs, ds, ds_x, pars_stored,\
                                                                     FIT_SIDEBAND_ONLY=FIT_SIDEBAND_ONLY)

c1,xframe_bkg,ras_bkg = ft.plot_background_after_fit(x,ds_x,total,sub_funcs)

In [ ]:
pars_stored

# Signal



In [ ]:
'''
total = ROOT.RooExtendPdf("fit_func", "Extended function for signal", sub_funcs["sig_pdf"], pars_d["nsig"])

if verbose:
    total.Print("v")

df_tmp1 = None
mask = None

if BNC_bool:
    # BNC
    mask = (df_sp['used_in_bkg_train'] == False)
    mask &= mask & (df_sp['used_in_sig_train'] == False)
    mask &= mask & (df_sp['cut_2']==True) & (df_sp['cut_3']==True) & (df_sp['cut_4']==True)
    mask &= mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
    mask &= mask & (df_sp['proba']>max_proba_cut)
    
    mask_sp = (df_sp['spmode']=='-999')
    df_tmp1 = df_sp[mask & mask_sp]
else:
    # BNV
    mask = (df_sp['used_in_sig_train'] == False)
    mask &= mask & (df_sp['cut_-1']==True)
    mask &= mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
    mask &= mask & (df_sp['proba']>max_proba_cut)
    
    mask_sp = (df_sp['spmode']=='-999')
    df_tmp1 = df_sp[mask & mask_sp]

#mask_sp = (df_sp['spmode']=='-999')
#df_tmp2 = df_sp[mask & mask_sp].sample(n=4)

#df_tmp = pd.concat([df_tmp1, df_tmp2])
df_tmp = df_tmp1

#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

ds_x = ds.reduce(x)

ds_x.Print("v")
'''


In [ ]:
'''
def fit_signal(pars_d, sub_funcs, ds_x, verbose=False, pars_stored=None):

    total = ROOT.RooExtendPdf("fit_func", "Extended function for signal", sub_funcs["sig_pdf"], pars_d["nsig"])
    
    nevents = ds_x.sumEntries()
    print(f"nevents: {nevents}")
    
    pars_d["nCB"].setVal(1.0)
    pars_d["nCB"].setConstant(True)
    
    pars_d["conv_factor_calc"].setVal(1)
    pars_d["conv_factor_calc"].setConstant(True)
    
    pars_d["conv_factor_fit"].setVal(1)
    pars_d["conv_factor_fit"].setConstant(True)
    
    pars_d["conv_factor_err"].setVal(500)
    pars_d["conv_factor_err"].setConstant(True)
    
    pars_d["branching_fraction"].setVal(nevents)
    pars_d["branching_fraction"].setConstant(False)
    
    pars_d["meanCB"].setVal(5.279)
    pars_d["meanCB"].setConstant(False)
    
    pars_d["sigmaCB"].setVal(0.0028)
    pars_d["sigmaCB"].setConstant(False)
    
    pars_d["alphaCB"].setVal(2.000)
    pars_d["alphaCB"].setConstant(False)
    
    fit_results = total.fitTo(ds_x, Save=True, PrintLevel=-1)
    fit_results.Print("v")

    # To check stuff
    #mynsig = pars_d["conv_factor_fit"].getVal() * pars_d["branching_fraction"].getVal()
    #print(f"mynsig: {mynsig}    nsig: {pars_d['nsig'].getVal()}")
    
    fit_results.Print("v")

    #####################################
    # Save the values
    #####################################
    if pars_stored is None:
        pars_stored = {}
    pars_stored['alphaCB'] = {'val':pars_d['alphaCB'].getVal(), 'err':pars_d['alphaCB'].getError()}
    pars_stored['meanCB'] = {'val':pars_d['meanCB'].getVal(), 'err':pars_d['meanCB'].getError()}
    pars_stored['sigmaCB'] = {'val':pars_d['sigmaCB'].getVal(), 'err':pars_d['sigmaCB'].getError()}
    pars_stored['nsig'] = {'val':pars_d["nsig"].evaluate(), 'err':0.0}

    print("\nFrom fit to signal SP....\n")
    for val in pars_stored.keys():
        print(f"{val:12s}: {pars_stored[val]['val']:8.4f} +/- {pars_stored[val]['err']:.4f}")
    print()

    # To check
    #pars_d['nsig'].Print("v")
    #print(f'nsig (evaluate) {pars_d["nsig"].evaluate():.2f}')
    #print()

    fit_results.Print('v')
    
    return pars_stored, total, fit_results
'''

In [ ]:
x,y,ds_sig,ds_x_sig = ft.prepare_dataset(df_sp, max_proba_cut=info['max_proba_cut'], \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                             SIG_BKG_DATA='sig')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

pars_stored, total, fit_results_sig_only = ft.fit_signal(pars_d, sub_funcs, ds_x_sig, verbose=False, pars_stored=pars_stored)


In [ ]:
pars_stored

In [ ]:
#fit_results_sig_only.Print("v")
#fit_results_sig_only.Print('v')
#fit_results.Print('v')

In [ ]:
'''
def plot_signal_after_fit(x, ds_x, total, sub_funcs):
    
    xframe_sig = x.frame(ROOT.RooFit.Title("Signal"),  ROOT.RooFit.Bins(200))  # RooPlot
    
    c2 = ROOT.TCanvas("fit_sig_only", "fit_sigonly", 600, 300)#"fit", "fit", 600, 600)
    
    ds_x.plotOn(xframe_sig)
    
    total.plotOn(xframe_sig, ROOT.RooFit.Normalization(1.0, ROOT.RooAbsReal.RelativeExpected))
    
    ## Overlay the signal component
    ras_sig = ROOT.RooArgSet(sub_funcs['sig_pdf'])
    total.plotOn(
        xframe_sig, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
            ROOT.kDotted), ROOT.RooFit.Normalization(
                1.0, ROOT.RooAbsReal.RelativeExpected))
    
    xframe_sig.Draw()
    c2.Draw()
    
    return c2,xframe_sig,ras_sig
''';

In [ ]:
proba_cut = info['max_proba_cut']
#proba_cut = 0.0

x,y,ds,ds_x = ft.prepare_dataset(df_sp, max_proba_cut=proba_cut, \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                             SIG_BKG_DATA='sig')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

pars_stored, total, fit_results_bkg = ft.fit_signal(pars_d, sub_funcs, ds_x, pars_stored=pars_stored)

c2,xframe_sig,ras_sig = ft.plot_signal_after_fit(x,ds_x,total,sub_funcs)

In [ ]:
pars_stored


# Full fit

In [ ]:
print(f'max_proba_cut: {info["max_proba_cut"]= :6.4f}')
print(f"sig_eff:       {info['sig_eff']= :6.4f}\n")


In [ ]:
# Get the figure of merit
decay = 'BNV'
if BNC_bool:
    decay = 'BNC'

# FIX THIS SO IT CALCULATES IT FOR THE PROBA CUT
sig_eff = info['sig_eff']
proba_cut = info['max_proba_cut']
# NOT WORKING!
#proba_cut = 0.0
conv_factors, conv_factor_errs = bat.calculate_conversion_factor(df_sp, df_col, \
                                                                 decay=decay, \
                                                                 proba_cut=proba_cut, \
                                                                 region_definitions=region_definitions, \
                                                                 sig_eff=sig_eff)

conv_factor_calc = conv_factors[0]
conv_factor_err = conv_factor_errs[0]

info['conv_factor_calc'] = conv_factor_calc
info['conv_factor_err'] = conv_factor_err

print()
#####################################################################
#df_sp = pd.read_parquet(infilename_sp)
#df_col = pd.read_parquet(infilename_col)

print(info)

In [ ]:
#proba_cut = info['max_proba_cut']
proba_cut = 0.5

verbosity=-1

########################################################################
# Fit full background to get some good starting parameters
########################################################################

x,y,ds,ds_x = ft.prepare_dataset(df_sp, max_proba_cut=proba_cut, \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                               SIG_BKG_DATA='bkg')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

FIT_SIDEBAND_ONLY=False

pars_stored = {}
pars_stored, total, fit_results_bkg, reduced_data_x = ft.fit_background(x, pars_d, sub_funcs, ds, ds_x, verbose=verbosity, \
                                                                        pars_stored=pars_stored,\
                                                                     FIT_SIDEBAND_ONLY=FIT_SIDEBAND_ONLY)

c1,xframe_bkg,ras_bkg = ft.plot_background_after_fit(x,ds_x,total,sub_funcs)

print(pars_stored)

########################################################################
# Fit full signal to get some good starting parameters
########################################################################

x,y,ds_sig,ds_x_sig = ft.prepare_dataset(df_sp, max_proba_cut=proba_cut, \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                             SIG_BKG_DATA='sig')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

pars_stored, total, fit_results_sig_only = ft.fit_signal(pars_d, sub_funcs, ds_x_sig, verbose=verbosity, pars_stored=pars_stored)

c2,xframe_sig,ras_sig = ft.plot_signal_after_fit(x,ds_x_sig,total,sub_funcs)

toy_signal_data = total.generate(x, 1000)


print(pars_stored)


##########################################################################
# Fit the collision data or something similar
##########################################################################

x,y,ds,ds_x = ft.prepare_dataset(df_col, max_proba_cut=proba_cut, \
                               BNC_bool=False, \
                               region_definitions=region_definitions, \
                               SIG_BKG_DATA='data')

mypars,sub_funcs_list,total,pars_d,sub_funcs = ft.prepare_fitting_materials(x)

#print(pars_d)
#print()
#print(pars_stored)

# Add signal
nsig_to_add = 10
nvalues = int(toy_signal_data.sumEntries())
print(nvalues)

for n in range(nsig_to_add):
    values = toy_signal_data.get(n)
    temp_x = values[0].getVal()
    print(temp_x)
    x.setVal(temp_x)
    ds_x.add(ROOT.RooArgSet(x))
#x.setVal(5.279)
#ds_x.add(ROOT.RooArgSet(x))
#x.setVal(5.275)
#ds_x.add(ROOT.RooArgSet(x))
#x.setVal(5.282)
#ds_x.add(ROOT.RooArgSet(x))

#print(pars_stored)

pars_stored['cutoff'] = {}
pars_stored['cutoff']['val'] = 5.287

modelc,fconstraint,pars_stored, fit_results = ft.fit_signal_and_background(pars_d, sub_funcs, ds_x, total, \
                                                                                 verbose=verbosity, pars_stored=pars_stored,\
                                                                                info=info)

c4,xframe_unblinded_collision,ras_sig,ras_bkg = ft.plot_signal_and_background(x, ds_x, total, modelc, sub_funcs)






In [ ]:
print("\nFrom fit to signal and background collision data....\n")
for val in pars_stored.keys():
    print(f"{val:24s}: {pars_stored[val]['val']:8.4e} +/- {pars_stored[val]['err']:.4e}")
print()

for key,val in info.items():
    print(f'{key:24s} {val:8.4e}')


In [ ]:
tag = f'{proba_cut:.2f}_ML'

c1.Draw()
c1.SaveAs(f'SP_bkg_{tag}.png')

c2.Draw()
c2.SaveAs(f'SP_sig_{tag}.png')

#c3.Draw()
#c3.SaveAs(f'blinded_collision_data_{tag}.png')

c4.Draw()

In [ ]:
'''
mes_lo, mes_hi = 5.2,5.3

# Trying this for BNV mode
#mes_lo, mes_hi = 5.21,5.3
#deltae_lo, deltae_hi = -0.07,0.07

# BNC
#deltae_lo, deltae_hi = -0.2,0.2
#deltae_lo, deltae_hi = -0.05,0.05
deltae_lo, deltae_hi = -0.07,0.07

x,y = pdfs.build_xy([[mes_lo, mes_hi], [deltae_lo, deltae_hi]])

x.setRange("FULL",mes_lo,mes_hi)
y.setRange("FULL",deltae_lo, deltae_hi)

x.setRange("SIGNAL",5.25, mes_hi)
y.setRange("SIGNAL",-0.10, 0.1)

# Sideband 1 region
x.setRange("SB1", mes_lo,  mes_hi)
y.setRange("SB1", 0.075, deltae_hi)

# Sideband 2 region
x.setRange("SB2",  mes_lo,  mes_hi)
y.setRange("SB2", deltae_lo, -0.075)

# Sideband 3 region
x.setRange("SB3",  mes_lo,  5.27)
#x.setRange("SB3",  mes_lo,  5.25) # TESTING
#y.setRange("SB3", -0.075, 0.075)
y.setRange("SB3", deltae_lo, deltae_hi)

##########################################################
ds_x = ds.reduce(x)
##########################################################

mypars = []

dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x)#, use_double_cb, workspace)

mypars += dum_pars

total.Print("v")

##########################################################

# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f

###########################################################

for p in pars_d.keys():
    print(f"{p} ---------------------")
    #pars_d[p].Print("v")

###########################################################
''';

In [ ]:
#'''
#############################################################
# Collision
#############################################################

#max_proba_cut = max_proba_cut_org
#max_proba_cut = 0.0

'''
df_for_fit = df_col

mask = None

if BNC_bool:
    # BNC
    mask = (df_for_fit['cut_2']==True) & (df_for_fit['cut_3']==True) & (df_for_fit['cut_4']==True)
    mask &= mask & (df_for_fit['BpostFitDeltaE']<deltae_hi) & (df_for_fit['BpostFitDeltaE']>deltae_lo)
    mask &= mask & (df_for_fit['proba']>max_proba_cut)
else:
    # BNV
    mask = (df_for_fit['cut_-1']==True)
    mask &= mask & (df_for_fit['BpostFitDeltaE']<deltae_hi) & (df_for_fit['BpostFitDeltaE']>deltae_lo)
    mask &= mask & (df_for_fit['proba']>max_proba_cut)

df_tmp = df_for_fit[mask]
#df_tmp = df_tmp1
'''

'''
#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

ds_x = ds.reduce(x)

ds_x.Print("v")
'''
;

In [ ]:
'''
mypars = []
dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x)#, use_double_cb, workspace)
mypars += dum_pars
total.Print("v")

# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f
'''

# THIS WORKS IF WE EDIT IT!
'''
###########################################################
# Fit sideband first
###########################################################
reduced_data = ds.reduce(ROOT.RooFit.CutRange("SB3"))
reduced_data_x = reduced_data.reduce(x)
nentries_reduced = reduced_data_x.sumEntries()


# Background
#pars_d["nbkg"].setVal(ds_x.sumEntries())
pars_d["nbkg"].setVal(nentries_reduced)

pars_d["argpar"].setVal(argpar)
pars_d["cutoff"].setVal(cutoff)

pars_d["nbkg"].setConstant(False)
pars_d["argpar"].setConstant(False)
pars_d["cutoff"].setConstant(False)

## Signal

# BNC
pars_d["conv_factor_calc"].setVal(conv_factor_calc)
pars_d["conv_factor_fit"].setVal(conv_factor_calc)
pars_d["conv_factor_err"].setVal(conv_factor_err)
#pars_d["branching_fraction"].setVal(2.0e-7)
# SIDEBAND
pars_d["branching_fraction"].setVal(0)

pars_d["nCB"].setVal(1.0)
#pars_d["meanCB"].setVal(5.279)
#pars_d["sigmaCB"].setVal(2.7035e-03)
#pars_d["alphaCB"].setVal(16.27)
pars_d["meanCB"].setVal(meanCB)
pars_d["sigmaCB"].setVal(sigmaCB)
pars_d["alphaCB"].setVal(alphaCB)

pars_d["nCB"].setConstant(True)
pars_d["conv_factor_calc"].setConstant(True)
pars_d["conv_factor_err"].setConstant(True)
#pars_d["conv_factor_fit"].setConstant(False)
#pars_d["branching_fraction"].setConstant(False)
# Sideband
pars_d["conv_factor_fit"].setConstant(True)
pars_d["branching_fraction"].setConstant(True)

pars_d["meanCB"].setConstant(True)
pars_d["sigmaCB"].setConstant(True)
pars_d["alphaCB"].setConstant(True)

# All, sidebands first

fconstraint = ROOT.RooGaussian("fconstraint", "fconstraint", pars_d['conv_factor_fit'], pars_d['conv_factor_calc'].getVal(), pars_d['conv_factor_err'].getVal())
modelc = ROOT.RooProdPdf("modelc", "model with constraint", [total, fconstraint])

total.Print("v")
print()
modelc.Print("v")


fit_results = total.fitTo(reduced_data_x, Save=True, PrintLevel=-1, Range='SB3' )
#fit_results = modelc.fitTo(reduced_data_x, Save=True, PrintLevel=-1, Range='SB3' )

fit_results.Print("v")



'''
;

In [ ]:
#fit_results.Print("v")

'''
print()
print('------------------------------')
print()

print(f'{alphaCB = :.6f}    {alphaCB_err = :.4e}')
print(f'{meanCB = :.6f}    {meanCB_err = :.4e}')
print(f'{sigmaCB = :.6f}    {sigmaCB_err = :.4e}')


print(f'{argpar = :.6f}      {argpar_err = :.4e}')
print(f'{cutoff = :.6f}      {cutoff_err = :.4e}')
'''
;

In [ ]:
'''
print(f'{sig_eff = }\n')

br =     pars_d['branching_fraction'].getVal()
br_err = pars_d['branching_fraction'].getError()

print(f'{br = :.3e}     {br_err = :.3e}\n')

pars_d['nsig'].Print("v")

print(pars_d["nsig"].evaluate())
'''


In [ ]:
'''
#del c
c3 = ROOT.TCanvas("fit_blinded_collision", "fit_blinded_collision", 600, 300)#"fit", "fit", 600, 600)
# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe_collision = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(50))

#ds_x.plotOn(xframe_collision)
reduced_data_x.plotOn(xframe_collision)

total.plotOn(xframe_collision, ROOT.RooFit.Normalization(
    1.0))#, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
total.plotOn(
    xframe_collision, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0))#, ROOT.RooAbsReal.RelativeExpected))

# Draw the frame on the canvas

ROOT.gPad.SetLeftMargin(0.15)
xframe_collision.GetYaxis().SetTitleOffset(1.4)
xframe_collision.Draw()

c3.Draw()

c3.SaveAs("testfit_collision.png")
'''
;


In [ ]:
###########################################################
# Fit full
###########################################################
'''
def fit_signal_and_background(pars_d, sub_funcs, ds_x, verbose=-1, pars_stored=None):

    print("About to fit!")
    # Background
    pars_d["nbkg"].setVal(ds_x.sumEntries())
    pars_d["argpar"].setVal(pars_stored['argpar']['val'])
    pars_d["cutoff"].setVal(pars_stored['cutoff']['val'])
    
    pars_d["nbkg"].setConstant(False)
    pars_d["argpar"].setConstant(False)
    pars_d["cutoff"].setConstant(False)
    
    ## Signal
    
    # BNC
    pars_d["conv_factor_calc"].setVal(conv_factor_calc)
    pars_d["conv_factor_fit"].setVal(conv_factor_calc)
    pars_d["conv_factor_err"].setVal(conv_factor_err)
    pars_d["branching_fraction"].setVal(2.0e-7)
    
    pars_d["nCB"].setVal(1.0)
    pars_d["meanCB"].setVal(pars_stored['meanCB']['val'])
    pars_d["sigmaCB"].setVal(pars_stored['sigmaCB']['val'])
    pars_d["alphaCB"].setVal(pars_stored['alphaCB']['val'])
    #pars_d["meanCB"].setVal(meanCB)
    #pars_d["sigmaCB"].setVal(sigmaCB)
    #pars_d["alphaCB"].setVal(alphaCB)
    
    pars_d["nCB"].setConstant(True)
    pars_d["conv_factor_calc"].setConstant(True)
    pars_d["conv_factor_err"].setConstant(True)
    pars_d["conv_factor_fit"].setConstant(False)
    pars_d["branching_fraction"].setConstant(False)
    # Sideband
    #pars_d["conv_factor_fit"].setConstant(True)
    #pars_d["branching_fraction"].setConstant(True)
    
    pars_d["meanCB"].setConstant(True)
    pars_d["sigmaCB"].setConstant(True)
    pars_d["alphaCB"].setConstant(True)
    
    # All, sidebands first
    
    fconstraint = ROOT.RooGaussian("fconstraint", "fconstraint", pars_d['conv_factor_fit'], pars_d['conv_factor_calc'].getVal(), pars_d['conv_factor_err'].getVal())
    modelc = ROOT.RooProdPdf("modelc", "model with constraint", [total, fconstraint])
    
    total.Print("v")
    print()
    modelc.Print("v")
    
    #fit_results = total.fitTo(ds_x, Save=True, PrintLevel=-1 )
    fit_results = modelc.fitTo(ds_x, Save=True, PrintLevel=verbose )
    
    fit_results.Print("v")

    for val in pars_stored.keys():
        if val=='nsig':
            continue
        pars_stored[val] = {'val':pars_d[val].getVal(), 'err':pars_d[val].getError()}
    pars_stored['nsig'] = {'val':pars_d["nsig"].evaluate(), 'err':0.0}
    for val in ['conv_factor_fit', 'conv_factor_calc', 'conv_factor_err', 'branching_fraction']:
        pars_stored[val] = {'val':pars_d[val].getVal(), 'err':pars_d[val].getError()}

    return total,modelc,fconstraint,pars_stored, fit_results
''';

In [ ]:
pars_stored

In [ ]:
'''
def plot_signal_and_background(x, ds_x, total, sub_funcs):
    c4 = ROOT.TCanvas("fit_unblinded_collision", "fit_blinded_collision", 600, 300)#"fit", "fit", 600, 600)
    # Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
    # rather than observed number of events (==data.numEntries())
    xframe_unblinded_collision = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(50))
    
    ds_x.plotOn(xframe_unblinded_collision)
    
    modelc.plotOn(xframe_unblinded_collision, ROOT.RooFit.Normalization(
        1.0))#, ROOT.RooAbsReal.RelativeExpected))
    
    # Overlay the background component of model with a dashed line
    ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
    modelc.plotOn(
        xframe_unblinded_collision, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
            ROOT.kDashed), ROOT.RooFit.Normalization(
                1.0))#, ROOT.RooAbsReal.RelativeExpected))
    
    # Overlay the background component of model with a dashed line
    ras_sig = ROOT.RooArgSet(sub_funcs['sig_pdf'])
    modelc.plotOn(
        xframe_unblinded_collision, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
            ROOT.kDashed), ROOT.RooFit.LineColor("r") , ROOT.RooFit.Normalization(
                1.0))#, ROOT.RooAbsReal.RelativeExpected))
    
    # Draw the frame on the canvas
    
    ROOT.gPad.SetLeftMargin(0.15)
    xframe_unblinded_collision.GetYaxis().SetTitleOffset(1.4)
    xframe_unblinded_collision.Draw()
    
    c4.Draw()
    
    c4.SaveAs("testfit_collision.png")

    return c4,xframe_unblinded_collision,ras_sig,ras_bkg
    
''';

In [ ]:
pars_stored

In [ ]:
nvalues = int(ds_x.sumEntries())
print(nvalues)

for n in range(nvalues):
    values = ds_x.get(n)
    print(values[0].getVal())

In [ ]:
tag = f'{proba_cut:.2f}_ML'

c1.Draw()
c1.SaveAs(f'SP_bkg_{tag}.png')

c2.Draw()
c2.SaveAs(f'SP_sig_{tag}.png')

#c3.Draw()
#c3.SaveAs(f'blinded_collision_data_{tag}.png')

c4.Draw()

# MC study

In [ ]:

# Create manager
# ---------------------------
 
# Instantiate ROOT.RooMCStudy manager on model with x as observable and given choice of fit options
#
# The Silence() option kills all messages below the PROGRESS level, only a single message
# per sample executed, any error message that occur during fitting
#
# The Extended() option has two effects:
#    1) The extended ML term is included in the likelihood and
#    2) A poisson fluctuation is introduced on the number of generated events
#
# The FitOptions() given here are passed to the fitting stage of each toy experiment.
# If Save() is specified, fit result of each experiment is saved by the manager
#
# A Binned() option is added in self example to bin the data between generation and fitting
# to speed up the study at the expemse of some precision


val = pars_d['branching_fraction']
#val.setVal(1e-7)
val.setVal(5e-7)

mcstudy = ROOT.RooMCStudy(
    #total,
    modelc,
    {x},
    Constrain='conv_factor_fit',
    Binned=True,
    Silence=True,
    Extended=True,
    FitOptions=dict(Save=True, PrintEvalErrors=0),
)
 
# Generate and fit events
# ---------------------------------------------
 
# Generate and fit 1000 samples of Poisson(nExpected) events
mcstudy.generateAndFit(1000)#, 100)
 
# Explore results of study
# ------------------------------------------------
 
# Make plots of the distributions of mean, error on mean and the pull of
# mean
frame1 = mcstudy.plotParam(val, Bins=40)
frame2 = mcstudy.plotError(val, Bins=40)
frame3 = mcstudy.plotPull(val, Bins=40, FitGauss=True)
 
# Plot distribution of minimized likelihood
frame4 = mcstudy.plotNLL(Bins=40)
 
# Make some histograms from the parameter dataset
#hh_cor_a0_s1f = mcstudy.fitParDataSet().createHistogram("hh", a1, YVar=sig1frac)
#hh_cor_a0_a1 = mcstudy.fitParDataSet().createHistogram("hh", a0, YVar=a1)
 
# Access some of the saved fit results from individual toys
#corrHist000 = mcstudy.fitResult(0).correlationHist("c000")
#corrHist127 = mcstudy.fitResult(127).correlationHist("c127")
#corrHist953 = mcstudy.fitResult(953).correlationHist("c953")
 
# Draw all plots on a canvas
ROOT.gStyle.SetPalette(1)
ROOT.gStyle.SetOptStat(0)
c = ROOT.TCanvas("rf801_mcstudy", "rf801_mcstudy", 900, 900)
c.Divide(3, 3)
c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
frame1.GetYaxis().SetTitleOffset(1.4)
frame1.Draw()
c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
frame2.GetYaxis().SetTitleOffset(1.4)
frame2.Draw()
c.cd(3)
ROOT.gPad.SetLeftMargin(0.15)
frame3.GetYaxis().SetTitleOffset(1.4)
frame3.Draw()
c.cd(4)
ROOT.gPad.SetLeftMargin(0.15)
frame4.GetYaxis().SetTitleOffset(1.4)
frame4.Draw()

#c.cd(5)
#ROOT.gPad.SetLeftMargin(0.15)
#hh_cor_a0_s1f.GetYaxis().SetTitleOffset(1.4)
#hh_cor_a0_s1f.Draw("box")

#c.cd(6)
#ROOT.gPad.SetLeftMargin(0.15)
#hh_cor_a0_a1.GetYaxis().SetTitleOffset(1.4)
#hh_cor_a0_a1.Draw("box")

'''
c.cd(7)
ROOT.gPad.SetLeftMargin(0.15)
corrHist000.GetYaxis().SetTitleOffset(1.4)
corrHist000.Draw("colz")
c.cd(8)
ROOT.gPad.SetLeftMargin(0.15)
corrHist127.GetYaxis().SetTitleOffset(1.4)
corrHist127.Draw("colz")
c.cd(9)
ROOT.gPad.SetLeftMargin(0.15)
corrHist953.GetYaxis().SetTitleOffset(1.4)
corrHist953.Draw("colz")
'''

c.SaveAs("rf801_mcstudy.png")
 
# Make ROOT.RooMCStudy object available on command line after
# macro finishes
ROOT.gDirectory.Add(mcstudy)



In [ ]:
c.Draw()

In [ ]:
# Make plot of distribution of generated value of f parameter
h_conv_factor_fit_gen = mcstudy.fitParDataSet().createHistogram("conv_factor_fit", ROOT.RooFit.AutoBinning(40));

# Make plot of distribution of fitted value of f parameter
frame_cff_1 = mcstudy.plotParam(pars_d['conv_factor_fit'], ROOT.RooFit.Bins(40));
frame_cff_1.SetTitle("Distribution of fitted f values");


# Make plot of pull distribution on f
frame_cff_2 = mcstudy.plotPull(pars_d['conv_factor_fit'], ROOT.RooFit.Bins(40), ROOT.RooFit.FitGauss());
frame_cff_2.SetTitle("Distribution of f pull values");


#'''
c_cff = ROOT.TCanvas("rf804_mcstudy_constr", "rf804_mcstudy_constr", 1200, 400);
c_cff.Divide(3);
c_cff.cd(1);
ROOT.gPad.SetLeftMargin(0.15);
h_conv_factor_fit_gen.GetYaxis().SetTitleOffset(1.4);
h_conv_factor_fit_gen.Draw();

c_cff.cd(2);
ROOT.gPad.SetLeftMargin(0.15);
frame_cff_1.GetYaxis().SetTitleOffset(1.4);
frame_cff_1.Draw();

c_cff.cd(3);
ROOT.gPad.SetLeftMargin(0.15);
frame_cff_2.GetYaxis().SetTitleOffset(1.4);
frame_cff_2.Draw();
#'''

c_cff.Draw()

;

In [ ]:
#mcstudy.fitParDataSet().c

In [ ]:
pars_d['nbkg'].getVal()

In [ ]:
for i in range(10):
    vtemp =  mcstudy.fitResult(i)
    
    p = vtemp.floatParsFinal()
    
    v = p.getRealValue("nbkg")

    print(v)

In [ ]:
vtemp.Print("v")

In [ ]:
mcstudy

# Generating toys

In [ ]:
dataset_information= pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:

bkg_spmodes= ["998","1005","3981","1235","1237"]
sig_spmodes= ["-999"]

spmodes= bkg_spmodes+sig_spmodes

weights= {}
for sp in spmodes: 
    weights[sp]= bat.scaling_value(int(sp),dataset_information=dataset_information, cs_data= cs_data, plot= False, verbose= False)
    #weights[sp]=1

weights["-999"]= 1000 #scales signal higher 
weights["0"]= 1 #idk what this is for;;; ASK

print(weights)


print(weights['998'])